In [38]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
# Desired input shape for VGG16
input_shape = (224, 224, 3)

In [40]:
# ImageDataGenerator for rescaling and resizing and data_augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to the range [0, 1]
    validation_split=0.2,  # Optional: Split data into training and validation sets
    preprocessing_function=lambda x: tf.image.resize(x, (input_shape[0], input_shape[1])),
    rotation_range=20,  # Rotate images by a random angle within the specified range
    width_shift_range=0.2,  # Shift images horizontally by a fraction of total width
    height_shift_range=0.2,  # Shift images vertically by a fraction of total height
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Adjust brightness
#   contrast_range=[0.5, 1.5],  # Adjust contrast
    channel_shift_range=50,  # Shift channel values
    fill_mode='nearest'  # Fill in newly created pixels after rotation or shifting
)

In [41]:
# Path to your dataset
dataset_path = "C:/Users/user_99/Desktop/data_of_parikar"

In [42]:
# Data generators for training and validation
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # For training data
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # For validation data
)

Found 10239 images belonging to 12 classes.
Found 2555 images belonging to 12 classes.


In [43]:
# Pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# Freeze the convolutional base, unfreeze the last 3 convolutional layers
#Fine_tunning
for layer in base_model.layers[:-3]:
    layer.trainable = False

In [45]:
# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(12, activation='softmax')) # Final Dense layer with 12 nodes (number of classes) and softmax activation

In [46]:
# Compile the model
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [47]:
# Model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               6422784   
                                                                 
 dense_5 (Dense)             (None, 12)                3084      
                                                                 
Total params: 21140556 (80.64 MB)
Trainable params: 11145484 (42.52 MB)
Non-trainable params: 9995072 (38.13 MB)
_________________________________________________________________


In [48]:
# Model train
model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
320/320 [==============================] - 624s 2s/step - loss: 1.1758 - accuracy: 0.6172 - val_loss: 0.8536 - val_accuracy: 0.7194
Epoch 2/10
320/320 [==============================] - 609s 2s/step - loss: 0.7583 - accuracy: 0.7593 - val_loss: 0.8326 - val_accuracy: 0.7256
Epoch 3/10
320/320 [==============================] - 611s 2s/step - loss: 0.6418 - accuracy: 0.7965 - val_loss: 0.5894 - val_accuracy: 0.8078
Epoch 4/10
320/320 [==============================] - 611s 2s/step - loss: 0.5834 - accuracy: 0.8121 - val_loss: 0.5363 - val_accuracy: 0.8278
Epoch 5/10
320/320 [==============================] - 610s 2s/step - loss: 0.5325 - accuracy: 0.8328 - val_loss: 0.5951 - val_accuracy: 0.8200
Epoch 6/10
320/320 [==============================] - 610s 2s/step - loss: 0.5153 - accuracy: 0.8405 - val_loss: 0.5982 - val_accuracy: 0.8215
Epoch 7/10
320/320 [==============================] - 611s 2s/step - loss: 0.4962 - accuracy: 0.8463 - val_loss: 0.6574 - val_accuracy: 0.8121

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\user_99\AppData\Local\Temp\tmp5bgek5ud\assets


INFO:tensorflow:Assets written to: C:\Users\user_99\AppData\Local\Temp\tmp5bgek5ud\assets


In [52]:
tflite_model_path = "C:/Users/user_99/Desktop/tflite_model.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)